<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


In [5]:
response = requests.get(static_url)
soup = BeautifulSoup(response.content, 'html.parser')


In [6]:
launch_tables = soup.find_all('table', {'class': 'wikitable'})


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [7]:
# use requests.get() method with the provided static_url
# URL de la página de Wikipedia archivada
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Realizar solicitud HTTP GET
response = requests.get(static_url)

# Verificar que la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    print("Página solicitada correctamente.")
else:
    print(f"Error al solicitar la página: Código {response.status_code}")


# assign the response to a object
# URL estática de la página de Wikipedia
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Realizar la solicitud HTTP GET y asignar a un objeto llamado 'response'
response = requests.get(static_url)

# Imprimir el código de estado para verificar éxito
print(f"Código de estado de la respuesta: {response.status_code}")


Página solicitada correctamente.
Código de estado de la respuesta: 200


In [8]:
import requests

# Static URL of the Falcon 9 and Falcon Heavy launches Wikipedia snapshot (June 9, 2021)
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform HTTP GET request
response = requests.get(static_url)

# Check response status code
if response.status_code == 200:
    print("Página solicitada correctamente.")
else:
    print(f"Error al solicitar la página: Código {response.status_code}")

# Optionally: print status code again for confirmation
print(f"Código de estado de la respuesta: {response.status_code}")


Página solicitada correctamente.
Código de estado de la respuesta: 200


Create a `BeautifulSoup` object from the HTML `response`


In [10]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
# Importar BeautifulSoup si aún no está importado
from bs4 import BeautifulSoup

# Crear el objeto BeautifulSoup usando el contenido de la respuesta
soup = BeautifulSoup(response.text, 'html.parser')

# Verificar (opcional): imprimir las primeras líneas del HTML
print(soup.prettify()[:500])  # Imprime los primeros 500 caracteres del HTML de forma legible


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vect


In [11]:
from bs4 import BeautifulSoup

# Crear el objeto BeautifulSoup usando el contenido HTML de la respuesta
soup = BeautifulSoup(response.text, 'html.parser')

# (Opcional) Imprimir los primeros 500 caracteres del HTML formateado para verificar
print(soup.prettify()[:500])


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vect


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [12]:
# Use soup.title attribute
# Obtener el título de la página
page_title = soup.title

# Mostrar el título (etiqueta completa)
print(page_title)

# Mostrar solo el texto dentro de la etiqueta <title>
print(page_title.string)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [13]:
# Obtener el título de la página
page_title = soup.title

# Mostrar la etiqueta completa <title>
print(page_title)

# Mostrar solo el texto dentro de la etiqueta <title>
print(page_title.string)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [14]:
import requests
from bs4 import BeautifulSoup

# 1. URL de la página wiki (puedes cambiarla por la que necesites)
url = "https://en.wikipedia.org/wiki/SpaceX"

# 2. Descargar el contenido HTML
response = requests.get(url)

# 3. Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# 4. Encontrar todas las tablas en la página
tables = soup.find_all('table')

print(f"Se encontraron {len(tables)} tablas en la página.")

# 5. Extraer los nombres de columnas de la primera tabla (por ejemplo)
first_table = tables[0]

# Extraer los headers <th> dentro de la tabla
headers = first_table.find_all('th')

# Obtener texto limpio de cada header
column_names = [header.get_text(strip=True) for header in headers]

print("Nombres de columnas extraídos:")
print(column_names)


Se encontraron 19 tablas en la página.
Nombres de columnas extraídos:
['Trade name', 'Company type', 'Industry', 'Founded', 'Founder', 'Headquarters', 'Key people', 'Products', 'Revenue', 'Operating income', 'Owner', 'Number of employees', 'Subsidiaries', 'Website']


In [15]:
import requests
from bs4 import BeautifulSoup

# URL snapshot for Falcon 9 and Falcon Heavy launches
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Download the HTML content
response = requests.get(static_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables with class 'wikitable' (typical for Wikipedia data tables)
tables = soup.find_all('table', {'class': 'wikitable'})

print(f"Se encontraron {len(tables)} tablas wikitable en la página.")

# Select the first wikitable as an example (or choose the right one based on inspection)
first_table = tables[0]

# Extract all header <th> cells in the first table
headers = first_table.find_all('th')

# Extract clean text for each header
column_names = [header.get_text(strip=True) for header in headers]

print("Nombres de columnas extraídos de la primera tabla:")
print(column_names)


Se encontraron 13 tablas wikitable en la página.
Nombres de columnas extraídos de la primera tabla:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


In [16]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
from bs4 import BeautifulSoup
import requests

# URL de ejemplo (puedes cambiarla)
url = 'https://en.wikipedia.org/wiki/SpaceX'

# Hacer la petición HTTP
response = requests.get(url)

# Parsear el contenido HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Usar find_all para obtener todas las tablas
tables = soup.find_all('table')

print(f"Cantidad de tablas encontradas: {len(tables)}")


# Assign the result to a list called `html_tables`
from bs4 import BeautifulSoup
import requests

url = 'https://en.wikipedia.org/wiki/SpaceX'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar todas las tablas y asignarlas a html_tables
html_tables = soup.find_all('table')

print(f"Se encontraron {len(html_tables)} tablas en la página.")


Cantidad de tablas encontradas: 19
Se encontraron 19 tablas en la página.


In [17]:
from bs4 import BeautifulSoup
import requests

# URL de ejemplo (puedes cambiarla por la que necesites)
url = 'https://en.wikipedia.org/wiki/SpaceX'

# Realizar la solicitud HTTP GET
response = requests.get(url)

# Parsear el contenido HTML con BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar todas las tablas y asignarlas a html_tables
html_tables = soup.find_all('table')

print(f"Se encontraron {len(html_tables)} tablas en la página.")


Se encontraron 19 tablas en la página.


Starting from the third table is our target table contains the actual launch records.


In [21]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table><tbody><tr>
<td class="sidebar-person-title-image" style="background-color:lavender;color:inherit;"><span class="mw-image-border" typeof="mw:File"><a class="mw-file-description" href="/wiki/File:Elon_Musk,_2018_(cropped).jpg"><img class="mw-file-element" data-file-height="1155" data-file-width="817" decoding="async" height="106" src="//upload.wikimedia.org/wikipedia/commons/thumb/0/06/Elon_Musk%2C_2018_%28cropped%29.jpg/120px-Elon_Musk%2C_2018_%28cropped%29.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/0/06/Elon_Musk%2C_2018_%28cropped%29.jpg/250px-Elon_Musk%2C_2018_%28cropped%29.jpg 2x" width="75"/></a></span></td>
<td class="sidebar-person-title" style="background-color:lavender;color:black;"><div><span class="tmp-color" style="color:black">This article is part of<br/>a series about</span></div><span class="vcard"><span class="fn"><a href="/wiki/Elon_Musk" title="Elon Musk"><span class="tmpl-colored-link" style="color: black; text-decoration: inherit;">Elon Musk<

In [22]:
# Select the third table (index 2) which contains launch records
first_launch_table = html_tables[2]

# Print the HTML content of the table (it might be long)
print(first_launch_table.prettify()[:1000])  # Print first 1000 chars neatly formatted


<table>
 <tbody>
  <tr>
   <td class="sidebar-person-title-image" style="background-color:lavender;color:inherit;">
    <span class="mw-image-border" typeof="mw:File">
     <a class="mw-file-description" href="/wiki/File:Elon_Musk,_2018_(cropped).jpg">
      <img class="mw-file-element" data-file-height="1155" data-file-width="817" decoding="async" height="106" src="//upload.wikimedia.org/wikipedia/commons/thumb/0/06/Elon_Musk%2C_2018_%28cropped%29.jpg/120px-Elon_Musk%2C_2018_%28cropped%29.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/0/06/Elon_Musk%2C_2018_%28cropped%29.jpg/250px-Elon_Musk%2C_2018_%28cropped%29.jpg 2x" width="75"/>
     </a>
    </span>
   </td>
   <td class="sidebar-person-title" style="background-color:lavender;color:black;">
    <div>
     <span class="tmp-color" style="color:black">
      This article is part of
      <br/>
      a series about
     </span>
    </div>
    <span class="vcard">
     <span class="fn">
      <a href="/wiki/Elon_Musk" tit

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [30]:
column_names = []
import requests
from bs4 import BeautifulSoup

def extract_column_from_header(row):
    # Eliminar etiquetas <sup> (notas, referencias)
    for sup in row.find_all('sup'):
        sup.extract()
    # Obtener texto limpio con espacios entre fragmentos y sin espacios extras
    column_name = row.get_text(separator=" ", strip=True)
    # Filtrar nombres vacíos o solo números
    if column_name == '' or column_name.isdigit():
        return None
    else:
        return column_name

# URL de ejemplo
url = 'https://en.wikipedia.org/wiki/SpaceX'

# Obtener contenido HTML
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar todas las tablas
html_tables = soup.find_all('table')

# Tomar la primera tabla para extraer headers
first_table = html_tables[0]

# Encontrar todos los elementos <th> en la tabla
th_elements = first_table.find_all('th')

# Lista para almacenar nombres de columnas
column_names = []

# Extraer nombre de cada columna usando la función
for th in th_elements:
    col_name = extract_column_from_header(th)
    if col_name:
        column_names.append(col_name)

# Mostrar resultado
print(column_names)


# Apply find_all() function with `th` element on first_launch_table
th_elements = first_launch_table.find_all('th')


# Asumiendo que first_launch_table es una tabla BeautifulSoup
th_elements = first_launch_table.find_all('th')

print(f"Se encontraron {len(th_elements)} elementos <th> en la tabla.")


# Asumiendo que first_launch_table ya es un objeto BeautifulSoup que representa la tabla

th_elements = first_launch_table.find_all('th')

print(f"Se encontraron {len(th_elements)} elementos <th> en la tabla.")


# Iterate each th element and apply the provided extract_column_from_header() to get a column name
column_names = []

for th in th_elements:
    col_name = extract_column_from_header(th)
    if col_name:  # Ignorar valores None o vacíos
        column_names.append(col_name)

print(column_names)


# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
column_names = []

for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print(column_names)


['Trade name', 'Company type', 'Industry', 'Founded', 'Founder', 'Headquarters', 'Key people', 'Products', 'Revenue', 'Operating income', 'Owner', 'Number of employees', 'Subsidiaries', 'Website']
Se encontraron 17 elementos <th> en la tabla.
Se encontraron 17 elementos <th> en la tabla.
['Flight No.', 'Date and time ( UTC )', 'Version, Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']
['Flight No.', 'Date and time ( UTC )', 'Version, Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


In [31]:
import requests
from bs4 import BeautifulSoup

def extract_column_from_header(row):
    for sup in row.find_all('sup'):
        sup.extract()
    column_name = row.get_text(separator=" ", strip=True)
    if column_name == '' or column_name.isdigit():
        return None
    else:
        return column_name

# URL snapshot Falcon 9 launches
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

response = requests.get(static_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar tablas wikitable
html_tables = soup.find_all('table', {'class': 'wikitable'})
print(f"Se encontraron {len(html_tables)} tablas wikitable.")

# Seleccionar la tabla correcta (ejemplo: la tercera tabla)
first_launch_table = html_tables[2]

# Extraer headers <th>
th_elements = first_launch_table.find_all('th')
print(f"Se encontraron {len(th_elements)} elementos <th> en la tabla.")

column_names = []
for th in th_elements:
    col_name = extract_column_from_header(th)
    if col_name:
        column_names.append(col_name)

print("Columnas extraídas:")
print(column_names)


Se encontraron 13 tablas wikitable.
Se encontraron 17 elementos <th> en la tabla.
Columnas extraídas:
['Flight No.', 'Date and time ( UTC )', 'Version, Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


In [36]:
def extract_column_from_header(row):
    # Remove <sup> tags (footnotes/references)
    for sup in row.find_all('sup'):
        sup.extract()
    # Get clean text with spaces between fragments, strip leading/trailing whitespace
    column_name = row.get_text(separator=" ", strip=True)
    # Filter out empty strings or pure digits
    if column_name == '' or column_name.isdigit():
        return None
    else:
        return column_name

# Extract all <th> elements from the launch table
th_elements = first_launch_table.find_all('th')

column_names = []

for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print("Column names extracted:")
print(column_names)


Column names extracted:
['Flight No.', 'Date and time ( UTC )', 'Version, Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


In [38]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)
soup = BeautifulSoup(response.content, 'html.parser')
html_tables = soup.find_all('table', {'class':'wikitable'})
first_launch_table = html_tables[2]  # for example, if that's the launch table


In [40]:
print(column_names)


['Flight No.', 'Date and time ( UTC )', 'Version, Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


Check the extracted column names


In [43]:
print(column_names)

['Flight No.', 'Date and time ( UTC )', 'Version, Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [46]:
# Inicializar el diccionario con listas vacías para cada columna
launch_dict = {col: [] for col in column_names}

# Si quieres eliminar una columna irrelevante, usa el nombre exacto (ejemplo: 'Date and time ( UTC )')
# Pero cuidado: si esa columna es relevante, no la elimines. Aquí la dejamos.

# Renombrar la columna para que coincida con las claves que usarás después (opcional)
if 'Version, Booster' in launch_dict:
    launch_dict['Version Booster'] = launch_dict.pop('Version, Booster')

# Añadir columnas extra que quieras, inicializándolas vacías
extra_cols = ['Date', 'Time']
for col in extra_cols:
    launch_dict[col] = []

# Ahora launch_dict tiene una lista vacía para cada columna necesaria

print(launch_dict)


{'Flight No.': [], 'Date and time ( UTC )': [], 'Launch site': [], 'Payload': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launch outcome': [], 'Booster landing': [], 'Version Booster': [], 'Date': [], 'Time': []}


In [47]:
# Eliminar la columna solo si existe, sin lanzar error
launch_dict.pop('Date and time ( )', None)


In [53]:
removed = launch_dict.pop('Date and time ( )', None)
if removed is None:
    print("La clave no existía.")
else:
    print("Clave eliminada.")


La clave no existía.


In [54]:
# Mostrar todas las claves para verificar el nombre exacto
print("Claves actuales en launch_dict:")
for key in launch_dict.keys():
    print(repr(key))

# Eliminar la clave 'Date and time ( )' solo si existe
launch_dict.pop('Date and time ( )', None)


Claves actuales en launch_dict:
'Flight No.'
'Date and time ( UTC )'
'Launch site'
'Payload'
'Payload mass'
'Orbit'
'Customer'
'Launch outcome'
'Booster landing'
'Version Booster'
'Date'
'Time'


In [56]:
# Supongamos que ya hiciste:
response = requests.get(static_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar todas las tablas 'wikitable'
html_tables = soup.find_all('table', {'class': 'wikitable'})

# Seleccionar la tabla de lanzamientos (tercera tabla)
launch_table = html_tables[2]

# Ahora puedes extraer columnas
columns = launch_table.find_all('th')
column_names = [extract_column_from_header(col) for col in columns]
column_names = [col for col in column_names if col]


In [58]:
# Suponiendo que launch_table es tu tabla BeautifulSoup

# Extraer nombres de columnas limpiando etiquetas y valores vacíos
columns = launch_table.find_all('th')
column_names = [extract_column_from_header(col) for col in columns]
column_names = [col for col in column_names if col]  # Filtrar None

# Crear diccionario con listas vacías para cada columna
launch_dict = {col: [] for col in column_names}

# Eliminar columna irrelevante, si existe (ajusta el nombre según el correcto)
launch_dict.pop('Date and time ( )', None)  # En tu caso probablemente no existe

# Inicializar columnas específicas con listas vacías (puedes omitir si ya existen)
for key in ['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 
            'Customer', 'Launch outcome']:
    if key not in launch_dict:
        launch_dict[key] = []

# Agregar columnas personalizadas si no existen aún
for key in ['Version Booster', 'Booster landing', 'Date', 'Time']:
    if key not in launch_dict:
        launch_dict[key] = []



In [60]:
print("Claves y tamaño de listas en launch_dict:")
for key, value in launch_dict.items():
    print(f"{key}: {len(value)} elementos")


Claves y tamaño de listas en launch_dict:
Flight No.: 0 elementos
Date and time ( UTC ): 0 elementos
Version, Booster: 0 elementos
Launch site: 0 elementos
Payload: 0 elementos
Payload mass: 0 elementos
Orbit: 0 elementos
Customer: 0 elementos
Launch outcome: 0 elementos
Booster landing: 0 elementos
Version Booster: 0 elementos
Date: 0 elementos
Time: 0 elementos


In [62]:
# Asegúrate de haber ejecutado esto antes
tables = soup.find_all('table', class_='wikitable')

# Asumimos que la segunda tabla es la de lanzamientos (puedes ajustar el índice si no es correcta)
launch_table = tables[1]  # O prueba con tables[2], depende del contenido

# Extraer encabezados de la tabla
columns = launch_table.find_all('th')
column_names = [extract_column_from_header(col) for col in columns]
column_names = [col for col in column_names if col]  # Eliminar valores None

# Verifica los nombres de columnas encontrados
print("Columnas extraídas:", column_names)


Columnas extraídas: ['Flight No.', 'Date and time ( UTC )', 'Version, Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


In [63]:
# Crear diccionario con listas vacías para cada columna
launch_dict = {col: [] for col in column_names}

# Renombrar columna para facilitar uso posterior
launch_dict['Version Booster'] = launch_dict.pop('Version, Booster')

# Agregar columnas personalizadas (si no están ya)
for extra in ['Date', 'Time']:
    if extra not in launch_dict:
        launch_dict[extra] = []


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [71]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata

# -------- Funciones auxiliares --------
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out

def landing_status(table_cells):
    out = [i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        idx = mass.find("kg")
        if idx != -1:
            new_mass = mass[:idx + 2]
        else:
            new_mass = "0"
    else:
        new_mass = "0"
    return new_mass

# -------- Web scraping --------
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)
soup = BeautifulSoup(response.content, 'html.parser')

# -------- Inicializar el diccionario --------
launch_dict = {
    'Flight No.': [],
    'Date': [],
    'Time': [],
    'Version Booster': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Booster landing': []
}

# -------- Parsear las tablas --------
extracted_row = 0
for table in soup.find_all('table', "wikitable plainrowheaders collapsible"):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
            else:
                flag = False
        else:
            flag = False

        row = rows.find_all('td')

        if flag:
            extracted_row += 1
            # Flight No.
            launch_dict['Flight No.'].append(flight_number)

            # Date and Time
            datatimelist = date_time(row[0])
            launch_dict['Date'].append(datatimelist[0].strip(','))
            launch_dict['Time'].append(datatimelist[1])

            # Version Booster
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string if row[1].a else row[1].text.strip()
            launch_dict['Version Booster'].append(bv)

            # Launch site
            launch_site = row[2].a.string if row[2].a else row[2].text.strip()
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string if row[3].a else row[3].text.strip()
            launch_dict['Payload'].append(payload)

            # Payload mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string if row[5].a else row[5].text.strip()
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string if row[6].a else row[6].text.strip()
            launch_dict['Customer'].append(customer)

            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# -------- Crear DataFrame --------
df = pd.DataFrame(launch_dict)
print(f"\nNúmero total de lanzamientos extraídos: {len(df)}")
print(df.head(10))  # Mostrar las primeras 10 filas



Número total de lanzamientos extraídos: 121
  Flight No.               Date   Time   Version Booster     Launch site  \
0          1        4 June 2010  18:45  F9 v1.07B0003.18           CCAFS   
1          2    8 December 2010  15:43  F9 v1.07B0004.18           CCAFS   
2          3        22 May 2012  07:44  F9 v1.07B0005.18           CCAFS   
3          4     8 October 2012  00:35  F9 v1.07B0006.18           CCAFS   
4          5       1 March 2013  15:10  F9 v1.07B0007.18           CCAFS   
5          6  29 September 2013  16:00    F9 v1.17B10038            VAFB   
6          7    3 December 2013  22:41           F9 v1.1           CCAFS   
7          8     6 January 2014  22:06           F9 v1.1           CCAFS   
8          9      18 April 2014  19:25           F9 v1.1  Cape Canaveral   
9         10       14 July 2014  15:15           F9 v1.1  Cape Canaveral   

                                Payload Payload mass        Orbit Customer  \
0  Dragon Spacecraft Qualification Unit 

In [73]:
import pandas as pd

df = pd.DataFrame(launch_dict)
print(df.head())  # Mostrar las primeras filas


  Flight No.             Date   Time   Version Booster Launch site  \
0          1      4 June 2010  18:45  F9 v1.07B0003.18       CCAFS   
1          2  8 December 2010  15:43  F9 v1.07B0004.18       CCAFS   
2          3      22 May 2012  07:44  F9 v1.07B0005.18       CCAFS   
3          4   8 October 2012  00:35  F9 v1.07B0006.18       CCAFS   
4          5     1 March 2013  15:10  F9 v1.07B0007.18       CCAFS   

                                Payload Payload mass Orbit Customer  \
0  Dragon Spacecraft Qualification Unit            0   LEO   SpaceX   
1                                Dragon            0   LEO     NASA   
2                                Dragon       525 kg   LEO     NASA   
3                          SpaceX CRS-1     4,700 kg   LEO     NASA   
4                          SpaceX CRS-2     4,877 kg   LEO     NASA   

  Launch outcome Booster landing  
0      Success\n         Failure  
1        Success         Failure  
2        Success    No attempt\n  
3      Succe

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [75]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [76]:
import pandas as pd

# Supongamos que esta es tu lista con datos de la columna 'LandingPad'
landing_pad = ['LZ-1', None, 'OCISLY', 'LZ-4', None, None, 'JRTI', 'OCISLY', 'LZ-1', None]

# Crear un DataFrame
df = pd.DataFrame({'LandingPad': landing_pad})

# Contar valores faltantes usando isnull()
missing_count = df['LandingPad'].isnull().sum()

print("Valores faltantes en la columna 'LandingPad':", missing_count)


Valores faltantes en la columna 'LandingPad': 4


In [77]:
import requests
import pandas as pd

# Obtener todos los lanzamientos pasados
url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(url)
launches = response.json()

# Convertir a DataFrame
df = pd.json_normalize(launches)

# Obtener datos de los cohetes para identificar Falcon 1 y Falcon 9
rockets_url = "https://api.spacexdata.com/v4/rockets"
rockets_response = requests.get(rockets_url)
rockets = rockets_response.json()
rockets_df = pd.DataFrame(rockets)

# Crear un diccionario id->name para los cohetes
rocket_id_name = dict(zip(rockets_df['id'], rockets_df['name']))

# Añadir columna con el nombre del cohete al DataFrame de lanzamientos
df['rocket_name'] = df['rocket'].map(rocket_id_name)

# Filtrar solo lanzamientos de Falcon 9 (excluir Falcon 1)
falcon9_launches = df[df['rocket_name'] == 'Falcon 9']

print("Número de lanzamientos Falcon 9:", len(falcon9_launches))


Número de lanzamientos Falcon 9: 179


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.


In [83]:
import requests
import pandas as pd

# Hacer la solicitud GET a la API de lanzamientos
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

# Convertir la respuesta JSON a un DataFrame plano
data = pd.json_normalize(response.json())

# Ver la primera fila de la columna 'static_fire_date_utc'
static_fire_date = data['static_fire_date_utc'].iloc[0]

# Mostrar el valor original y el año
print(f"Fecha completa: {static_fire_date}")

# Extraer el año si no es None
if pd.notnull(static_fire_date):
    year = pd.to_datetime(static_fire_date).year
    print(f"Año: {year}")
else:
    print("No hay fecha de static fire en la primera fila.")


Fecha completa: 2006-03-17T00:00:00.000Z
Año: 2006


In [84]:
import requests
import pandas as pd

# Obtener todos los lanzamientos
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)
launches = response.json()

# Normalizar el JSON en un DataFrame plano
df = pd.json_normalize(launches)

# Los datos de los cohetes están en la columna 'rocket' como IDs, necesitamos obtener info de cohetes
# Obtener datos de cohetes
rockets_url = "https://api.spacexdata.com/v4/rockets"
rockets_response = requests.get(rockets_url)
rockets = rockets_response.json()
df_rockets = pd.DataFrame(rockets)

# Crear un diccionario para mapear id de cohete a nombre
rocket_id_name_map = dict(zip(df_rockets['id'], df_rockets['name']))

# Agregar columna con nombre de cohete al DataFrame de lanzamientos
df['rocket_name'] = df['rocket'].map(rocket_id_name_map)

# Filtrar lanzamientos que NO sean Falcon 1
df_falcon9 = df[df['rocket_name'] != 'Falcon 1']

# Contar lanzamientos de Falcon 9 (y otros distintos de Falcon 1)
count_falcon9 = df_falcon9.shape[0]

print(f"Número de lanzamientos de Falcon 9 (excluyendo Falcon 1): {count_falcon9}")


Número de lanzamientos de Falcon 9 (excluyendo Falcon 1): 200


In [90]:
print(df.columns.tolist())


['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window', 'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules', 'payloads', 'launchpad', 'flight_number', 'name', 'date_utc', 'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores', 'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused', 'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships', 'links.patch.small', 'links.patch.large', 'links.reddit.campaign', 'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery', 'links.flickr.small', 'links.flickr.original', 'links.presskit', 'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia', 'fairings']


In [91]:
import requests
import pandas as pd

url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)
launches = response.json()

df = pd.json_normalize(launches, sep='_')

# Extraer los valores 'landing_pad' dentro de la lista de 'cores' para cada lanzamiento
landing_pads = []

for cores_list in df['cores']:
    if cores_list:  # si no es None ni vacío
        # Extraer 'landing_pad' del primer core (puede haber más de uno)
        landing_pad = cores_list[0].get('landing_pad', None)
        landing_pads.append(landing_pad)
    else:
        landing_pads.append(None)

# Contar valores faltantes
missing_count = sum(pd.isna(landing_pads))
print(f"Número de valores faltantes en la columna 'landingPad' dentro de 'cores': {missing_count}")


Número de valores faltantes en la columna 'landingPad' dentro de 'cores': 205


In [92]:
import requests
import pandas as pd

url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)
launches = response.json()

df = pd.json_normalize(launches, sep='_')

landing_pads = []

for cores_list in df['cores']:
    if cores_list:
        landing_pad = cores_list[0].get('landing_pad', None)
        landing_pads.append(landing_pad)
    else:
        landing_pads.append(None)

landing_pads_series = pd.Series(landing_pads)
missing_count = landing_pads_series.isnull().sum()

print(f"Número de valores faltantes en la columna 'landingPad' dentro de 'cores': {missing_count}")


Número de valores faltantes en la columna 'landingPad' dentro de 'cores': 205
